In [1]:
import gc
import re
import string
import logging
from copy import copy, deepcopy

import underthesea
import numpy as np
import pyspark
from pyspark.sql import SparkSession, Row
import pyspark.sql.functions as f
from pyspark.sql.functions import col, udf, lit, greatest, monotonically_increasing_id, concat_ws
from pyspark.sql.types import StringType, IntegerType, FloatType, DoubleType, StructType, StructField, MapType, BooleanType
from pyspark.ml.feature import Tokenizer, HashingTF, IDF, MinHashLSH
from pyspark.ml.linalg import Vectors, VectorUDT

In [2]:
spark = SparkSession.builder.master("spark://172.24.0.11:7077").getOrCreate()
spark

24/01/03 12:50:53 WARN Utils: Your hostname, haihp02 resolves to a loopback address: 127.0.1.1; using 192.168.102.7 instead (on interface wlp5s0)
24/01/03 12:50:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/03 12:50:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
print(spark.sparkContext._jsc.sc().getExecutorMemoryStatus().keySet().size())
# spark.sparkContext.setLogLevel("DEBUG")
# logging.getLogger("py4j").setLevel("DEBUG")

4


In [3]:
hdfs_file_path = [
    'hdfs://192.168.102.7:10000/haihp02/real_estate_data/bds.jsonl',
    'hdfs://192.168.102.7:10000/haihp02/real_estate_data/i-batdongsan.jsonl',
    'hdfs://192.168.102.7:10000/haihp02/real_estate_data/nhadatviet.jsonl',
]
df = spark.read.json(hdfs_file_path)

In [7]:
df.describe().show()

+-------+--------------------+----------------+--------------------+----------+------------------+--------------------+-----------------+--------------------+
|summary|         description|     estate_type|                link| post_date|           post_id|               price|           square|               title|
+-------+--------------------+----------------+--------------------+----------+------------------+--------------------+-----------------+--------------------+
|  count|              250796|          240015|              250796|    250796|            250796|              248964|           247809|              250795|
|   mean|                NULL|            NULL|                NULL|      NULL|3759772.4077058644|6.948346463197141E14|16837.76337179037|                NULL|
| stddev|                NULL|            NULL|                NULL|      NULL|2288274.0038934583|1.409071832582140...| 5107081.44598701|                NULL|
|    min|                    |        Biệt thự